# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,561.399250,-0.650604,1234,560.748646,562.049853,1121.497292,1124.099707
6,174.898536,-0.780962,1234,174.117574,175.679498,348.235149,351.358996
7,952.628597,-0.596044,1234,952.032553,953.224641,1904.065106,1906.449282
8,612.083956,0.883836,1234,612.967791,611.200120,1225.935583,1222.400240
9,158.656050,0.126510,1234,158.782560,158.529540,317.565120,317.059080
10,298.921815,-0.059651,1234,298.862163,298.981466,597.724326,597.962932
11,196.451341,-0.892687,1234,195.558654,197.344028,391.117307,394.688055
12,711.674172,-0.440107,1234,711.234065,712.114279,1422.468130,1424.228558
13,773.164848,-0.989111,1234,772.175737,774.153959,1544.351473,1548.307919
14,282.420439,-0.905269,1234,281.515171,283.325708,563.030341,566.651416


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-560.748646,562.049853
6,-174.117574,175.679498
7,-952.032553,953.224641
8,-612.967791,611.200120
9,-158.782560,158.529540
10,-298.862163,298.981466
11,-195.558654,197.344028
12,-711.234065,712.114279
13,-772.175737,774.153959
14,-281.515171,283.325708


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-560.748646,562.049853
6,-174.117574,175.679498
7,-952.032553,953.224641
8,-612.967791,611.200120
9,-158.782560,158.529540
10,-298.862163,298.981466
11,-195.558654,197.344028
12,-711.234065,712.114279
13,-772.175737,774.153959
14,-281.515171,283.325708


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-560.74864597,  562.04985347],
       [-174.11757439,  175.67949815],
       [-952.03255305,  953.22464107],
       [-612.96779128,  611.20011996],
       [-158.78256018,  158.5295401 ],
       [-298.86216312,  298.98146589],
       [-195.55865372,  197.34402763],
       [-711.23406499,  712.11427879],
       [-772.17573651,  774.15395928],
       [-281.51517067,  283.32570811]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1119.497292,-1124.099707
6,346.235149,-351.358996
7,1902.065106,-1906.449282
8,1223.935583,-1222.400240
9,315.565120,-317.059080
10,595.724326,-597.962932
11,389.117307,-394.688055
12,1420.468130,-1424.228558
13,1542.351473,-1548.307919
14,561.030341,-566.651416


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.561399,0.174698
6,0.174899,0.109519
7,0.952629,0.201978
8,0.612084,0.941918
9,0.158656,0.563255
10,0.298922,0.470174
11,0.196451,0.053657
12,0.711674,0.279947
13,0.773165,0.005444
14,0.282420,0.047366


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,561.399250,-0.650604
6,174.898536,-0.780962
7,952.628597,-0.596044
8,612.083956,0.883836
9,158.656050,0.126510
10,298.921815,-0.059651
11,196.451341,-0.892687
12,711.674172,-0.440107
13,773.164848,-0.989111
14,282.420439,-0.905269


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
